In [1]:
!git clone https://github.com/tylerachang/word-acquisition-language-models/

Cloning into 'word-acquisition-language-models'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 187 (delta 91), reused 146 (delta 50), pack-reused 0
Receiving objects: 100% (187/187), 16.07 MiB | 17.58 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [2]:
!pip install numpy
!pip install scipy
!pip install torch
!pip install nltk
!pip install sentencepiece
!pip install tokenizers
!pip install transformers[torch]
!pip install spacy
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.2 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/UIUCLearningLanguageLab/AOCHILDES

  Cloning https://github.com/UIUCLearningLanguageLab/AOCHILDES to /tmp/pip-req-build-qdk2a7tn
  Running command git clone --filter=blob:none --quiet https://github.com/UIUCLearningLanguageLab/AOCHILDES /tmp/pip-req-build-qdk2a7tn
  Resolved https://github.com/UIUCLearningLanguageLab/AOCHILDES to commit e25348138282c35f4270800e592676b8713e67a1
  Preparing metadata (setup.py) ... done
  Created wheel for aochildes: filename=aochildes-3.0.0-py3-none-any.whl size=41322875 sha256=ebcf34aa67d10bb27cd0aa9a5ef203170c10b423027f2a24002511bb58feb13d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3x__l9_c/wheels/b7/9e/c5/cd013d6d40aa0309899f3232a4779e769b222d7bec7e8724d3
Successfully built aochildes


In [4]:
from aochildes.dataset import AOChildesDataSet
transcripts = AOChildesDataSet().load_transcripts()

Looking for transcripts in /usr/local/lib/python3.10/dist-packages/aochildes/original_transcripts
Utterances before dropping rows: 1,379,962
Utterances after  dropping rows: 1,180,889
Preparing AOCHILDES transcripts...
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


In [5]:
import re
for i, _ in enumerate(transcripts):
    transcripts[i] = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", transcripts[i]) # removing punctuation
    transcripts[i] = " ".join(transcripts[i].split()) # removing multiple spaces
    transcripts[i] = " " + transcripts[i]

In [6]:
import numpy as np
lens = np.array([len(t) for t in transcripts])
print(lens.mean())

5874.176150121066


In [7]:
print(len(transcripts))

3304


In [8]:
with open('./sample_data/ao_childes_train.txt', 'w+') as f:
    f.write("\n".join(transcripts[:-100]))

In [9]:
with open('./sample_data/ao_childes_eval.txt', 'w+') as f:
    f.write("\n".join(transcripts[-100:]))

In [10]:
!python3 ./word-acquisition-language-models/scripts/train_spm_tokenizer.py \
--input_file="./sample_data/ao_childes_train.txt" \
--output="./sample_data/spm" \
--vocab_size=512

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./sample_data/ao_childes_train.txt
  input_format: 
  model_prefix: ./sample_data/spm
  model_type: UNIGRAM
  vocab_size: 512
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 10000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 1
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0

In [11]:
!python3 ./word-acquisition-language-models/scripts/convert_spm_to_hf_tokenizer.py \
--input="./sample_data/spm.model" \
--output_dir="./sample_data/hf_tokenizer"

Using local sentencepiece model file as tokenizer.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1925: FutureWarning: Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
252 tokens added.
Final vocab size: 768 (this should be set in the language model configs).
Done.


In [12]:
!python3 ./word-acquisition-language-models/scripts/tokenize_dataset.py \
--tokenizer="./sample_data/hf_tokenizer" \
--input_file="./sample_data/ao_childes_train.txt" \
--output_file="./sample_data/ao_childes_tokenized.txt" \
--max_segments=2 \
--max_seq_len=-1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Tokenizing file: ./sample_data/ao_childes_train.txt
Finished tokenization: 1602 examples.


In [13]:
!python3 ./word-acquisition-language-models/scripts/tokenize_dataset.py \
--tokenizer="./sample_data/hf_tokenizer" \
--input_file="./sample_data/ao_childes_eval.txt" \
--output_file="./sample_data/ao_childes_eval_tokenized.txt" \
--max_segments=2 \
--max_seq_len=-1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Tokenizing file: ./sample_data/ao_childes_eval.txt
Finished tokenization: 50 examples.


In [29]:
## RNN bidirectional config
{
  "rnn_cell": "LSTM",
  "embedding_size": 256,
  "hidden_sizes": [256],
  "dropout": 0.1,
  "emb_tied": "standard",
  "rnn_mode": "bidirectional",
  "max_seq_len": 128
}

{'rnn_cell': 'LSTM',
 'embedding_size': 256,
 'hidden_sizes': [256],
 'dropout': 0.1,
 'emb_tied': 'standard',
 'rnn_mode': 'bidirectional',
 'max_seq_len': 128}

In [27]:
!python3 ./word-acquisition-language-models/lm_code/run_rnn_language_modeling.py \
--tokenizer_name="./sample_data/hf_tokenizer" \
--config_name="./word-acquisition-language-models/lm_configs/rnn_bidirectional_config.json" \
--do_train --do_eval \
--train_data_file="./sample_data/ao_childes_tokenized.txt" \
--eval_data_file="./sample_data/ao_childes_eval_tokenized.txt" \
--train_iterable \
--eval_iterable \
--per_device_train_batch_size=64 \
--per_device_eval_batch_size=64 \
--evaluation_strategy="steps" \
--eval_steps=1000 \
--save_steps=250 \
--max_steps=2000 \
--warmup_steps=500 \
--learning_rate=0.0001 --adam_epsilon=1e-6 --weight_decay=0.01 \
--output_dir="./models/rnn-bidirectional" \
--overwrite_output_dir

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Training model from scratch.
NUM TRAINABLE MODEL PARAMETERS: 28938240
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Counting examples in train file. This can be slow.
1602it [00:00, 13089.89it/s]
Finished counting: 1602 examples.
Counting examples in train file. This can be slow.
50it [00:00, 11432.98it/s]
Finished counting: 50 examples.
Note: 26 steps per epoch.
Starting epoch 0.
Example iterator complete.
Starting epoch 1.
Example iterator complete.
Starting epoch 2.
Example iterator complete.


In [28]:
!python3 ./word-acquisition-language-models/lm_code/word_evaluation.py \
--tokenizer="./sample_data/hf_tokenizer" \
--wordbank_file="./word-acquisition-language-models/r_code/tacl_data/child_data/child_aoa.tsv" \
--examples_file="./sample_data/ao_childes_tokenized.txt" \
--max_samples=512 \
--batch_size=64 \
--output_file="./sample_data/bilstm_surprisals_10k.txt" \
--model_dir="./models/rnn-bidirectional" --model_type="bilstm" \
--save_samples="./sample_data/bidirectional_samples.pickle"

Streaming output truncated to the last 5000 lines.
Finished inference.
Median rank: 47
Mean surprisal: 8.118330001831055
Stdev surprisal: 1.9141840934753418
Accuracy: 0.007352941203862429

Evaluation token: ▁push
22 / 247 tokens
CHECKPOINT STEP: 1750
Num examples: 66
Finished inference.
Median rank: 79
Mean surprisal: 9.657222747802734
Stdev surprisal: 1.9945831298828125
Accuracy: 0.0

Evaluation token: ▁say
23 / 247 tokens
CHECKPOINT STEP: 1750
Num examples: 329
Finished inference.
Median rank: 22
Mean surprisal: 6.908140182495117
Stdev surprisal: 1.8788493871688843
Accuracy: 0.015197568573057652

Evaluation token: ▁tape
24 / 247 tokens
CHECKPOINT STEP: 1750
Num examples: 90
Finished inference.
Median rank: 17
Mean surprisal: 7.166518211364746
Stdev surprisal: 2.834437131881714
Accuracy: 0.056179776787757874

Evaluation token: ▁cup
25 / 247 tokens
CHECKPOINT STEP: 1750
Num examples: 62
Finished inference.
Median rank: 47
Mean surprisal: 8.212102890014648
Stdev surprisal: 1.93686091899